#### EDA Libraries

In [44]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests
%matplotlib inline

In [70]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [71]:
response = requests.get(url)
print("Response Code: ", response)

Response Code:  <Response [200]>


### Let's Scrap Data from Website usin Beautiful Soup

In [72]:
import bs4 

In [73]:
soup = bs4.BeautifulSoup(response.content)

#### Let's Find Tables in this page

In [74]:
print(len(soup.find_all('table')))

3


##### There are 3 tables in this Content same as in pandas 

## Let's Access data cells in first table:

In [75]:
Postal_Code = []
Borough = []
Neighbourhood = []

In [76]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if len(cells) > 0:
        Postal_Code.append(cells[0].text.strip())
        Borough.append(cells[1].text.strip())
        Neighbourhood.append(cells[2].text.strip())

### `3.1 The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood`

In [77]:
data = pd.DataFrame({
    "PostalCode": Postal_Code,
    "Borough": Borough,
    "Neighborhood": Neighbourhood
})

In [78]:
data.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### let's look into data a little bit

In [79]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
PostalCode      180 non-null object
Borough         180 non-null object
Neighborhood    180 non-null object
dtypes: object(3)
memory usage: 4.3+ KB


#### We have total 180 Entries

### let's start with Data Exploring

### `3.2 Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.`

#### check out all cells with Not assigned Borough

In [80]:
data[data['Borough'] == 'Not assigned']

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
7,M8A,Not assigned,Not assigned
10,M2B,Not assigned,Not assigned
15,M7B,Not assigned,Not assigned
...,...,...,...
174,M4Z,Not assigned,Not assigned
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned


`we have 77 rows out of 180 having No Borough Information in them`

### Dropping all Cells having Bourough "Not assigned"

In [82]:
data = data[data['Borough'] != 'Not assigned'].reset_index(drop=True)

data.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


##### Let's check if some PostalCode and Boroguh are redudent

In [61]:
(data['PostalCode'].value_counts() > 1).sum()

0

In [62]:
(data["Borough"].value_counts() > 1).sum()

9

`There are total 9 unique Borough Exists in Table`

In [63]:
data["Borough"].value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
York                 5
East York            5
Mississauga          1
Name: Borough, dtype: int64

`Let's Group all neighborshood in the same borough`

### `3.3 More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.`

In [97]:
df = data.groupby(['PostalCode', 'Borough'], as_index=False).agg(lambda x: ",".join(x))

In [98]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [99]:
df.shape

(103, 3)

### `3.4 If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.`

let's check if some shell does not have neighborhood

In [100]:
(df['Neighborhood'] == 'Not assigned').sum()

0

In [101]:
(df['Borough'] == 'Not assigned').sum()

0

`It seems we does not have any Not assigned shell in both columns but it's good to check and write code to replace all negihbors with borough if neighbors have value Not assigned`

In [102]:
for index, row in df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']
        
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


`Final shape of Our Data`

In [103]:
toronto.shape

(103, 3)

#### Let's verify is our DataFrame same as required by the question

In [105]:
# create a new test dataframe
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

index_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in index_list:
    test_df = test_df.append(df[toronto["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill, Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford, Maryvale"
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."


In [106]:
df.to_csv('toronta_neighborhood.csv', index=False)